In [48]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import flika as flk
from fish.ephys.ephys import load, windowed_variance, estimate_onset
from scipy.ndimage.filters import gaussian_filter1d
from fish.image.zds import ZDS
from fish.util.fileio import to_dask
from dask.distributed import Client
from fish.util.distributed import get_cluster
from dask.array import coarsen
flk.start_flika()

Starting flika
flika menubar already initialized.
Starting flika inside IPython


In [49]:
def mydff(v):
    from fish.image.vol import dff
    camera_offset = 80
    window = 400
    percentile = 20
    offset = 10    
    downsample = 10;
    return dff((v - camera_offset).clip(1, None), window, percentile, offset, downsample, axis=0).astype('float32')


def shift_yx(im, shifts, block_id):
    from scipy.ndimage.interpolation import shift
    t = block_id[0]    
    return shift(im.astype('float32'), (0,0, *shifts[t]), order=1, cval=100)

In [50]:
base_dirs = ['/nrs/ahrens/Yu/newfromdm11/glia_neuron_imaging/20161012/20161012_1_1_cy221_6dpf_0gain_wholebrain_20161012_113040/', 
            '/nrs/ahrens/Yu/newfromdm11/glia_neuron_imaging/20161022/fish1/20161022_1_1_cy221_6dpf_GA_0gain_20161022_151003/',
            '/nrs/ahrens/Yu/newfromdm11/glia_neuron_imaging/20161206/fish1/20161206_1_2_gfap_gc6f_6dpf_CLpause_OLGU_20161206_123501/',
            '/nrs/ahrens/Yu/newfromdm11/glia_neuron_imaging/20161206/fish2/20161206_2_1_gfap_gc6f_6dpf_CLpause_OLGU_20161206_161111/',
            '/nrs/ahrens/Yu/newfromdm11/glia_neuron_imaging/20170111/fish1/20170111_1_1_gfapgc_6dpf_GAIN_TRIAL_20170111_123355/',
            '/nrs/ahrens/Yu/newfromdm11/glia_neuron_imaging/20170529/fish1/20170529_1_1_GFAPGC_0gain_20170529_152728/']

In [126]:
base_dir

'/groups/ahrens/ahrenslab/YuMu/SPIM/active_datasets/20171022/fish1/20171022_1_3_GFAPrgeco_injectedTRPV1_6dpf_beforeaftercsn10uM_20171022_141052/'

In [125]:
from skimage.io import imread
from scipy.ndimage.filters import median_filter
from functools import partial

base_dir = base_dirs[1] 
dset = ZDS(base_dir + 'raw/')
data = to_dask(dset.files)

shifts = -np.load(base_dir.replace('raw','proc') + '/reg/regparams_affine.npy')[:,:-1,-1]
shifts = median_filter(shifts, size=(300,1))
data.shape

(10464, 27, 772, 800)

In [12]:
# get giving up timepoints
from scipy.io import loadmat
from h5py import File
epd = File(base_dir + 'ephys/data.mat', 'r')['data']
gu_inds = loadmat(base_dir + 'ephys/GU_ind.mat')['GU_ind']
gu_inds_cam = epd['frame'][:].squeeze().astype('int')[gu_inds[:,1]]
window = np.arange(-20,20)

In [111]:
roi = (slice(None), slice(None), slice(None), slice(None))
data_tx = data.map_blocks(shift_yx, shifts[roi[0]], dtype='float32')
medfilt = partial(median_filter, size=(1,1,5,5))
data_filt = data_tx.map_blocks(medfilt)

ds_xy = 4
data_ds = coarsen(np.mean, data_tx, {2: ds_xy, 3: ds_xy})
rechunked = data_ds.rechunk(chunks=(data_ds.shape[0], data_ds.shape[1], 32,32))
data_dff = rechunked.map_blocks(mydff, dtype='float32')
data_dff_trial = data_dff.vindex[gu_inds_cam + window.reshape(-1,1)].mean(1)

In [65]:
cluster = get_cluster()
client = Client(cluster)
client

Client Scheduler: tcp://10.36.107.41:33519 Dashboard: http://10.36.107.41:37570/status,Cluster Workers: 0 Cores: 0 Memory: 0 B


In [112]:
%%time
import time
cluster.start_workers(200)
# give some time for the workers to arrive
time.sleep(20)
result_dff = data_dff.compute()
cluster.stop_workers(cluster.workers)

CPU times: user 5min 16s, sys: 4min 21s, total: 9min 37s
Wall time: 8min 56s


In [118]:
result_dff.shape

(7062, 24, 159, 176)

In [119]:
out_path = '/groups/ahrens/ahrenslab/projects/ngm/'
fname = dset.metadata['data_header'] + '_gu_ave.tif'
print(out_path + fname)
from skimage.io import imsave
imsave(out_path + fname, result_dff, imagej=True)

/groups/ahrens/ahrenslab/projects/ngm/20171107_1_3_gfaprgeco_gfaptrpv1_6dpf_csn.tif


/groups/ahrens/home/bennettd/anaconda3/lib/python3.6/site-packages/skimage/external/tifffile/tifffile.py:1088: UserWarning: truncating ImageJ file
  warnings.warn("truncating ImageJ file")


In [30]:
from fish.util.plot import proj_fuse

In [33]:
fused = np.array([proj_fuse(r, np.max, aspect=(8,1,1)) for r in result_dff])

/groups/ahrens/home/bennettd/anaconda3/lib/python3.6/site-packages/skimage/transform/_warps.py:110: UserWarning: Anti-aliasing will be enabled by default in skimage 0.15 to avoid aliasing artifacts when down-sampling images.
  warn("Anti-aliasing will be enabled by default in skimage 0.15 to "


In [34]:
flk.start_flika()
flk.window.Window(fused.transpose(0,2,1))

Starting flika
flika menubar already initialized.
Starting flika inside IPython
